In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import pyvista as pv
import numpy as np
import os

# ==========================================
# 1. ツール設定
# ==========================================
class Config:
    TO_MM = 1000.0
    TO_MPA = 1e-6         # Pa -> MPa
    NORM_DISP = 0.01      # 0.0057mm基準
    NORM_STRESS = 10.0    # 10.0MPa基準 (AIが1.0を学習しやすくするため)
    NORM_COORD = 100.0    
    TRAIN_LOAD = 1000.0   
    IN_CHANNELS = 5       
    OUT_CHANNELS = 4

# ==========================================
# 2. モデル定義 (精度重視)
# ==========================================
class BeamGNNTool(nn.Module):
    def __init__(self):
        super(BeamGNNTool, self).__init__()
        self.conv1 = GCNConv(Config.IN_CHANNELS, 128)
        self.conv2 = GCNConv(128, 128)
        
        # 形状（幾何学的な分布）を捉えるためのヘッド
        self.base_stress = nn.Sequential(
            nn.Linear(128, 128), nn.ReLU(),
            nn.Linear(128, 1) 
        )
        self.base_disp = nn.Sequential(
            nn.Linear(128, 128), nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # 荷重倍率を取得 (全ノードのうち、荷重がかかっている点の値を利用)
        # data.x[:, 4] に ratio が入っているので、その最大値を取得
        load_ratio = torch.max(data.x[:, 4]) 

        h = F.relu(self.conv1(x, edge_index))
        h = F.relu(self.conv2(h, edge_index))
        
        # 1000N時の「ベースの形」を予測
        raw_disp = self.base_disp(h)
        raw_stress = self.base_stress(h)
        
        # ★物理強制: 予測された形に「荷重倍率」を直接掛け算する
        out_disp = raw_disp * load_ratio
        out_stress = raw_stress * load_ratio
        
        return torch.cat([out_disp, out_stress], dim=1)

# ==========================================
# 3. データ処理 (ミーゼス優先ロジック)
# ==========================================
def mesh_to_graph(vtu_path, load_N):
    mesh = pv.read(vtu_path)
    mesh = mesh.cell_data_to_point_data() # セルデータをポイントに変換
    
    pos = mesh.points
    pos_norm = pos / Config.NORM_COORD
    
    z_min, z_max = pos[:, 2].min(), pos[:, 2].max()
    is_fixed = (pos[:, 2] <= z_min + 1e-5).astype(float)
    is_load_node = (pos[:, 2] >= z_max - 1e-5).astype(float)
    load_ratio = load_N / Config.TRAIN_LOAD
    load_feat = is_load_node * load_ratio
    
    x = torch.tensor(np.column_stack([pos_norm, is_fixed, load_feat]), dtype=torch.float)
    
    y = np.zeros((pos.shape[0], 4))
    keys = list(mesh.point_data.keys())
    
    # --- 応力キーの優先探索 ---
    # 1. 'mises' または 'von' を含むものを最優先
    # 2. 次に 'eqv' を含むもの
    # 3. それでもなければ 'stress' を含むもの (ただし 'tresca' は除外したい)
    s_key = None
    for k in keys:
        if any(target in k.lower() for target in ["mises", "von"]):
            s_key = k
            break
    if not s_key:
        for k in keys:
            if "eqv" in k.lower():
                s_key = k
                break
    
    # 変位キー
    u_key = next((k for k in keys if any(s in k.lower() for s in ["u", "displacement"])), None)

    if u_key:
        y[:, :3] = (mesh.point_data[u_key] * Config.TO_MM) / Config.NORM_DISP
        
    if s_key:
        s_val = mesh.point_data[s_key]
        if s_val.ndim > 1: s_val = np.linalg.norm(s_val, axis=1)
        y[:, 3] = (s_val * Config.TO_MPA) / Config.NORM_STRESS
        
        if load_N == Config.TRAIN_LOAD:
            print(f"--- 抽出成功 ---")
            print(f"  使用キー: {s_key}")
            print(f"  生応力最大: {np.max(s_val):.2e} Pa (={np.max(s_val)*1e-6:.1f} MPa)")
            print(f"  正規化ラベル値: {np.max(y[:, 3]):.4f} (1.0に近いほど良好)")
    else:
        print(f"警告: 適切な応力キーが見つかりません。利用可能: {keys}")

    edges = mesh.extract_all_edges().lines.reshape(-1, 3)[:, 1:]
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
    return Data(x=x, edge_index=edge_index, y=torch.tensor(y, dtype=torch.float))

# ==========================================
# 4. 学習・推論 (学習回数を増やして精度を追い込む)
# ==========================================
# 学習ループ内の重みと学習率を変更
def train_model_with_augmentation(vtu_file, model, device):
    # lrを 0.0002 に下げて精密に
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0002) 
    model.train()
    base_data = mesh_to_graph(vtu_file, 1000.0).to(device)
    # 比率を細かくして、中間をより覚えさせる
    load_ratios = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0] 
    
    print("\n[INFO] 最終微調整学習...")
    for epoch in range(8001):
        optimizer.zero_grad()
        total_loss = 0
        for ratio in load_ratios:
            current_data = base_data.clone()
            load_node_mask = (current_data.x[:, 4] > 0)
            current_data.x[load_node_mask, 4] = ratio
            current_data.y = base_data.y * ratio
            
            out = model(current_data)
            loss_disp = F.mse_loss(out[:, :3], current_data.y[:, :3])
            loss_stress = F.mse_loss(out[:, 3], current_data.y[:, 3])
            
            # 応力へのペナルティを 1000倍 に強化
            loss = loss_disp + (loss_stress * 1000.0) 
            loss.backward()
            total_loss += loss.item()
            
        optimizer.step()
        if epoch % 1000 == 0:
            print(f"Epoch {epoch:4d} | TotalLoss: {total_loss:.6f}")

# def run_inference(base_vtu, target_load_N, model):
#     data = mesh_to_graph(base_vtu, target_load_N).to(device)
#     model.eval()
#     with torch.no_grad():
#         pred = model(data).cpu().numpy()
    
#     disp_mm = pred[:, :3] * Config.NORM_DISP
#     stress_mpa = pred[:, 3] * Config.NORM_STRESS
    
#     print(f"\n--- [RESULT] 推論結果 ({target_load_N}N) ---")
#     print(f"最大変位予測: {np.max(np.abs(disp_mm)):.5f} mm")
#     print(f"最大応力予測: {np.max(stress_mpa):.5f} MPa")

# # --- 実行 ---
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = BeamGNNTool().to(device)
# train_file = "1_1000.vtu"
# if os.path.exists(train_file):
#     train_model_with_augmentation(train_file, model, device)
#     run_inference(train_file, 500.0, model)

def save_inference_to_vtu(base_vtu, pred, output_vtu,load_N):
    mesh = pv.read(base_vtu)
    mesh = mesh.cell_data_to_point_data()
    n_points = mesh.points.shape[0]
    if pred.shape[0] != n_points:
        raise ValueError(f"ノード数が一致しません: mesh={n_points}, pred={pred.shape[0]}")
    
    disp_mm = pred[:, :3] * Config.NORM_DISP
    stress_mpa = pred[:, 3] * Config.NORM_STRESS
    
    print(f"\n--- [RESULT] 推論結果 ({load_N}N) ---")
    print(f"最大変位予測: {np.max(np.abs(disp_mm)):.5f} mm")
    print(f"最大応力予測: {np.max(stress_mpa):.5f} MPa")


    # 変位と応力を追加
    mesh.point_data["gnn_disp"] = pred[:, :3] * Config.NORM_DISP
    mesh.point_data["gnn_stress"] = pred[:, 3] * Config.NORM_STRESS
    mesh.save(output_vtu)
    print(f"推論結果を {output_vtu} に保存しました")



# --- 実行 ---
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = BeamGNNTool().to(device)
# train_file = "1_1000.vtu"
# if os.path.exists(train_file):
#     train_model_with_augmentation(train_file, model, device)
#     # 推論
#     data = mesh_to_graph(train_file, 500.0).to(device)
#     model.eval()
#     with torch.no_grad():
#         pred = model(data).cpu().numpy()
#     # 保存
#     save_inference_to_vtu(train_file, pred, "gnn_500N_result.vtu")


def run_inference_and_save(base_vtu, model, device, load_N, output_vtu):
    data = mesh_to_graph(base_vtu, load_N).to(device)
    model.eval()
    with torch.no_grad():
        pred = model(data).cpu().numpy()
    save_inference_to_vtu(base_vtu, pred, output_vtu,load_N)

# --- 実行例 ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BeamGNNTool().to(device)
train_file = "1_1000.vtu"
if os.path.exists(train_file):
    train_model_with_augmentation(train_file, model, device)
    # 任意の荷重値で推論・保存
    run_inference_and_save(train_file, model, device, load_N=500.0, output_vtu="gnn_500N_result.vtu")
    run_inference_and_save(train_file, model, device, load_N=1800.0, output_vtu="gnn_1800N_result.vtu")

--- 抽出成功 ---
  使用キー: von Mises Stress
  生応力最大: 1.00e+07 Pa (=10.0 MPa)
  正規化ラベル値: 1.0000 (1.0に近いほど良好)

[INFO] 最終微調整学習...
Epoch    0 | TotalLoss: 2708.500230
Epoch 1000 | TotalLoss: 0.038836
Epoch 2000 | TotalLoss: 0.017040
Epoch 3000 | TotalLoss: 0.014502
Epoch 4000 | TotalLoss: 0.012709
Epoch 5000 | TotalLoss: 0.011467
Epoch 6000 | TotalLoss: 0.010476
Epoch 7000 | TotalLoss: 0.009741
Epoch 8000 | TotalLoss: 0.009098

--- [RESULT] 推論結果 (500.0N) ---
最大変位予測: 0.00249 mm
最大応力予測: 5.00702 MPa
推論結果を gnn_500N_result.vtu に保存しました

--- [RESULT] 推論結果 (1800.0N) ---
最大変位予測: 0.01212 mm
最大応力予測: 18.04735 MPa
推論結果を gnn_1800N_result.vtu に保存しました
